## Analysis of Agglomeration Economies in Europe

### author: "Erika Galletta"

#### date: "2023-06-21"


This is a notebook replicating the code used to clean and transform the data, fit the models and plot the results reported in the Thesis "Analysis of Agglomeration Economies in Europe" b Erika Galletta. The pdf of the thesis, expanding on the results and discussing the literature and the model selected can be found in the folder.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from tabulate import tabulate  # This will help create LaTeX-style tables

##### Cleaning and transforming the dataset

1. GVA Variable's columns (indicated in the indices'list) are scaled to express Millions
2. Year Variable's columns from 2005 to 2020 are created to represent GVA per population in the year
3. Log value of Year variables are added
4. Log Rank columns per year are added


In [ ]:
class Transform:
    def __init__(self, data: pd.DataFrame):
        self.dataset = data
        # Column number in the dataframes for GVA variable
        self.indices = [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]

    def formatGVA(self):
        self.dataset[self.indices] = self.dataset[self.indices] * 1000000
        return self.dataset

    def formatYear(self):
        for year in range(5, 21):  # Covers years 2005-2020
            gva_col = f"GVA{year:02d}"
            pop_col = f"pop{year:02d}"
            rank_col = f"rank{year:02d}"

            if gva_col in self.dataset.columns and pop_col in self.dataset.columns:
                self.dataset[f"Y{year:02d}"] = (
                    self.dataset[gva_col] / self.dataset[pop_col]
                )
                self.dataset[f"logY{year:02d}"] = np.log(self.dataset[f"Y{year:02d}"])

            if rank_col in self.dataset.columns:
                self.dataset[f"logrank{year:02d}"] = np.log(self.dataset[rank_col])

        return self.dataset

##### Define the models

Three models are defined. Given the preliminary results not reported here, only of the three is fitted for each country, as it is the only one with significant result. The explanation for this is rather straight forward, as geographical conformations are mutually exclusive in most cases.

1. Model 1: the explanatory variable considered is mount presence in the geographical area
2. Model 2: the explanatory variable considered is coast presence in the geographical area
3. Model 3: the only explanatory variable considered is Log Rank


In [ ]:
class Model:
    def __init__(self, dataset: pd.DataFrame):
        self.dataset = dataset

    def Model_1(self):
        models = {}
        for year in range(5, 21):  # Covers years 2005-2020
            logY_col = f"logY{year:02d}"
            logrank_col = f"logrank{year:02d}"

            if (
                logY_col in self.dataset.columns
                and logrank_col in self.dataset.columns
                and "MOUNT_TYPE" in self.dataset.columns
            ):
                formula = f"{logY_col} ~ {logrank_col} + MOUNT_TYPE"
                model = smf.ols(formula, data=self.dataset).fit()
                models[f"m{year:02d}"] = model

        return models

    def Model_2(self):
        models = {}
        for year in range(5, 21):  # Covers years 2005-2020
            logY_col = f"logY{year:02d}"
            logrank_col = f"logrank{year:02d}"

            if (
                logY_col in self.dataset.columns
                and logrank_col in self.dataset.columns
                and "MOUNT_TYPE" in self.dataset.columns
            ):
                formula = f"{logY_col} ~ {logrank_col} + COAST_TYPE"
                model = smf.ols(formula, data=self.dataset).fit()
                models[f"m{year:02d}"] = model

        return models

    def Model_3(self):
        models = {}
        for year in range(5, 21):  # Covers years 2005-2020
            logY_col = f"logY{year:02d}"
            logrank_col = f"logrank{year:02d}"

            if (
                logY_col in self.dataset.columns
                and logrank_col in self.dataset.columns
                and "MOUNT_TYPE" in self.dataset.columns
            ):
                formula = f"{logY_col} ~ {logrank_col}"
                model = smf.ols(formula, data=self.dataset).fit()
                models[f"m{year:02d}"] = model

        return models

    def execute(self):

        # Dictionary mapping model numbers to corresponding model functions
        models = {"1": self.Model_1, "2": self.Model_2, "3": self.Model_3}

        return models

##### Extract valuable insights from the models


In [ ]:
class Analysis:
    def __init__(self, models: dict, country_name=str):
        self.models = models
        self.country = country_name

    # Function to extract the adjusted R-squared values
    def extract_adj_r_squared(self):
        adj_r_squared = {}
        for year in range(5, 21):
            model_key = f"m{year:02d}"
            if model_key in self.models:
                adj_r_squared[year] = self.models[model_key].adj_rsquared
        return adj_r_squared

    # Function to extract coefficients: if the coefficient is positive, a zero is taken as per the model's theoretical logic
    def extract_coefficients(self):
        coefficients = {}
        for year in range(5, 21):
            model_key = f"m{year:02d}"
            if model_key in self.models:
                # Get the second coefficient (as per your original logic)
                coefficient = self.models[model_key].params[1]
                # If the coefficient is negative, store 0, otherwise store the negative value
                coefficients[year] = 0 if coefficient < 0 else -coefficient
        return coefficients

    # Function to plot coefficients
    def plot_coefficients(self, filename):
        coefficients = self.extract_coefficients()
        filename = "plot" + self.country + ".png"
        years = list(coefficients.keys())
        coeff_values = list(coefficients.values())

        plt.figure()
        plt.plot(years, coeff_values, marker="o")
        plt.xlabel("Year")
        plt.ylabel("Coefficient")
        plt.title(f"Change in Coefficient over Years {self.country}")
        plt.savefig(filename)
        plt.close()

        return coefficients

    def plot_radj(self, range: list):
        adj_r_squared = self.extract_adj_r_squared()

        # Print out the R-squared values
        for year, adj_r2 in adj_r_squared.items():
            print(f"Adjusted R-squared for {year}: {adj_r2}")

        y = []
        for year in range(range[0], range[-1]):
            y.append(f"m{year:02d}")

        radj = {key: adj_r_squared[key] for key in adj_r_squared if key in y}
        return self.generate_latex_table(
            radj,
            f"Adjusted R-squared, 2.b model, years 20{range[0]:02d}-20{range[-1]:02d}",
        )

    def generate_latex_table(df, caption):
        return tabulate(
            df,
            headers="keys",
            tablefmt="latex_booktabs",
            showindex=False,
            numalign="center",
            caption=caption,
        )

#### Fitting the models


In [ ]:
class Main:
    def __init__(self):
        # Initialize the countries and indices needed

        # Dictionary of countries name and the significant model number.
        # Some countries have None values as in those cases no model was significant
        self.country_names = {
            "Albania": "3",
            "Austria": None,
            "Belgium": "2",
            "Bulgaria": "3",
            "Czech Republic": "3",
            "Germany": None,
            "Denmark": "3",
            "Estonia": None,
            "Greece": "1",
            "Spain": None,
            "Finland": "2",
            "France": "2",
            "Croatia": None,
            "Hungary": "3",
            "Italy": "2",
            "Lithuania": "3",
            "Latvia": "3",
            "North Macedonia": "3",
            "Netherlands": "3",
            "Norway": None,
            "Poland": None,
            "Portugal": "3",
            "Romania": None,
            "Sweden": "3",
            "Slovenia": "2",
            "Slovakia": "3",
            "United Kingdom": None,
        }

        # List comprehension to filter out keys where the value is None
        self.valid_countries = [
            country
            for country, value in self.country_names.items()
            if value is not None
        ]

        # Calculate growth change for each country, handling missing coefficients
        self.growthchange = {
            "from 2005 to 2020": [],
            "from 2005 to 2012": [],
            "from 2012 to 2020": [],
        }

        self.yearly_comparison = {"2005": [], "2012": [], "2020": []}

        self.data = {}
        self.fitted_model = {}
        self.coefficients = {}

    def plotting_results(self, data: dict, title: str = None):
        for name, obs in data.items():
            # Create the bar plot
            plt.figure(figsize=(10, 6))
            plt.bar(self.valid_countries, obs, color="lightblue")
            plt.xlabel("Countries")
            plt.ylabel(f"Coefficients {title}")
            plt.title(
                f"{f"{title} in" if title else ""} Agglomeration Economies {name}"
            )
            plt.xticks(rotation=45)  # Rotate country labels for better readability
            plt.tight_layout()  # To make sure the labels fit nicely

            # Save the plot to a file
            plt.savefig(f"{f"{title}_" if title else ""}{name.replace(" ","_")}.png")

            # Optionally, show the plot
            plt.show()

    def model_fitting(self, CC, modelnumber):
        Model_ = Model(self.data[CC])
        Trasformation_ = Transform(self.data[CC])
        self.data[CC] = pd.read_excel("OFFICIAL_R_DATASET copy2.xlsx", sheet_name=CC)
        self.data[CC] = Trasformation_.formatGVA()
        self.data[CC] = Trasformation_.formatYear()
        models = Model_.execute()
        self.fitted_model[CC] = models[modelnumber](self.data[CC])
        # Extract adjusted R-squared values
        Analysis_ = Analysis(self.fitted_model[CC], CC)

        self.Radj_table_2005_2012 = Analysis_.plot_radj([5, 13])
        self.Radj_table_2012_2020 = Analysis_.plot_radj([12, 21])

        # Plot coefficients over the years
        self.coefficients[CC] = Analysis_.plot_coefficients()

        # Computing and plotting the growth change
        # If the country exists in the coefficients, calculate the growth change
        self.growthchange["from 2005 to 2020"].append(
            self.coefficients[CC][16] - self.coefficients[CC][1]
        )
        self.growthchange["from 2005 to 2012"].append(
            self.coefficients[CC][8] - self.coefficients[CC][1]
        )
        if CC != "North Macedonia":  # Incomplete time series
            self.growthchange["from 2012 to 2020"].append(
                self.coefficients[CC][16] - self.coefficients[CC][8]
            )

        # Computing and plotting the yearly results
        # If the country exists in the coefficients, calculate the growth change
        self.yearly_comparison["2005"].append(self.coefficients[CC][1])
        self.yearly_comparison["2012"].append(self.coefficients[CC][8])
        self.yearly_comparison["2020"].append(self.coefficients[CC][16])

    def execute(self):

        for CC, model_number in self.country_names.items():
            self.model_fitting(CC, model_number)

        self.plotting_results(self.valid_countries)
        self.plotting_results(self.growthchange, "Change")
        print(self.Radj_table_2005_2012, self.Radj_table_2012_2020)

#### Run code


In [ ]:
Main().execute()